## Loading Data

In [ ]:
import zipfile
with zipfile.ZipFile("/content/novozymes-enzyme-stability-prediction.zip", "r") as zip_ref:
  zip_ref.extractall("Data")

In [1]:
import pandas as pd
train_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train.csv")

In [2]:
train_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [3]:
train_updated_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train_updates_20220929.csv")
train_updated_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,69,NaN,NaN,NaN,NaN
1,70,NaN,NaN,NaN,NaN
2,71,NaN,NaN,NaN,NaN
3,72,NaN,NaN,NaN,NaN
4,73,NaN,NaN,NaN,NaN


## Updated CSV

In [ ]:
import pandas as pd


df_train = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train.csv", index_col="seq_id")
df_train_updates = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train_updates_20220929.csv", index_col="seq_id")

all_features_nan = df_train_updates.isnull().all("columns")

drop_indices = df_train_updates[all_features_nan].index
df_train = df_train.drop(index=drop_indices)

swap_ph_tm_indices = df_train_updates[~all_features_nan].index
df_train.loc[swap_ph_tm_indices, ["pH", "tm"]] = df_train_updates.loc[swap_ph_tm_indices, ["pH", "tm"]]

In [ ]:
df_train_updates

In [ ]:
df_train

In [2]:
df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train.csv")
updates = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train_updates_20220929.csv")

drop_seq_ids = updates[updates["pH"].isnull()]["seq_id"].values
change_seq_ids = updates[updates["pH"].notnull()]["seq_id"].values

# drop
df.drop(df[df.seq_id.isin(drop_seq_ids)].index, inplace=True)

# change
df.loc[df.seq_id.isin(change_seq_ids), ["pH", "tm"]] = updates[
    updates.seq_id.isin(change_seq_ids)
][["pH", "tm"]].values

df.to_csv("updated_train.csv", index=False)

In [ ]:
df_updated = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Notebooks/updated_train.csv")

In [ ]:
df_updated.head()

## EDA and Preprocessing

### BioPandas

In [7]:
import biopandas

In [8]:
somethin = ppdb.df["ATOM"]

NameError: name 'ppdb' is not defined

In [ ]:
wt = somethin.groupby('residue_number').b_factor.agg('first').values

### Continue

In [6]:
df_test = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/test.csv")
df_test.head()

,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


#### Test Data Pre processing

In [7]:
df_test.isnull().sum()

seq_id              0
protein_sequence    0
pH                  0
data_source         0
dtype: int64

In [8]:
df_test.drop("data_source", axis = 1, inplace = True)

In [3]:
df_updated = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Notebooks/updated_train.csv")
df_updated.drop("data_source", inplace = True, axis = 1)
df_updated.head()

,seq_id,protein_sequence,pH,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5


In [45]:
df_updated.isnull().sum()

seq_id                0
protein_sequence      0
pH                  286
tm                    0
dtype: int64

In [4]:
fillVal = df_updated[["pH"]].mean().round()
df_updated.fillna(fillVal, inplace = True)

In [5]:
df_updated.isnull().sum()

seq_id              0
protein_sequence    0
pH                  0
tm                  0
dtype: int64

### Encoding the Amino Acid Sequences

### Try One

##### Hashing Vectorizer

In [38]:
random_sequence = df_updated.iloc[1]["protein_sequence"]

In [31]:
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(n_features = 2**4)

X = vectorizer.fit_transform(random_sequence)

In [32]:
X

<1x16 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

#### TfidfVectorizer

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()

X_ = vec.fit_transform(random_sequence)

In [34]:
vec.get_feature_names_out()

array(['aaadgeplhneeeragagqvgrslpqeseeqrtgsrprrrrdlgsrlqaqrraqrvawedgdenvgqtvipaqeeegiekpaevhptgkigakklrkleekqarkaqreaeeaereerkrlesqreaewkkeeerlrlkeeqkeeeerkaqeeqarreheeylklkeafvveeegvsetmteeqshsfltefinyikkskvvlledlafqmglrtqdainriqdlltegtltgviddrgkfiyitpeelaavanfirqrgrvsitelaqasnsliswgqdlpaqas'],
      dtype=object)

### Try Two

#### List Strategy (Encoding singular letters)

In [9]:
import numpy as np

In [11]:
random_sequence

'AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSRLQAQRRAQRVAWEDGDENVGQTVIPAQEEEGIEKPAEVHPTGKIGAKKLRKLEEKQARKAQREAEEAEREERKRLESQREAEWKKEEERLRLKEEQKEEEERKAQEEQARREHEEYLKLKEAFVVEEEGVSETMTEEQSHSFLTEFINYIKKSKVVLLEDLAFQMGLRTQDAINRIQDLLTEGTLTGVIDDRGKFIYITPEELAAVANFIRQRGRVSITELAQASNSLISWGQDLPAQAS'

In [12]:
random_seq_list = np.array(list(random_sequence))

In [35]:
random_seq_list

array(['A', 'A', 'A', 'D', 'G', 'E', 'P', 'L', 'H', 'N', 'E', 'E', 'E',
       'R', 'A', 'G', 'A', 'G', 'Q', 'V', 'G', 'R', 'S', 'L', 'P', 'Q',
       'E', 'S', 'E', 'E', 'Q', 'R', 'T', 'G', 'S', 'R', 'P', 'R', 'R',
       'R', 'R', 'D', 'L', 'G', 'S', 'R', 'L', 'Q', 'A', 'Q', 'R', 'R',
       'A', 'Q', 'R', 'V', 'A', 'W', 'E', 'D', 'G', 'D', 'E', 'N', 'V',
       'G', 'Q', 'T', 'V', 'I', 'P', 'A', 'Q', 'E', 'E', 'E', 'G', 'I',
       'E', 'K', 'P', 'A', 'E', 'V', 'H', 'P', 'T', 'G', 'K', 'I', 'G',
       'A', 'K', 'K', 'L', 'R', 'K', 'L', 'E', 'E', 'K', 'Q', 'A', 'R',
       'K', 'A', 'Q', 'R', 'E', 'A', 'E', 'E', 'A', 'E', 'R', 'E', 'E',
       'R', 'K', 'R', 'L', 'E', 'S', 'Q', 'R', 'E', 'A', 'E', 'W', 'K',
       'K', 'E', 'E', 'E', 'R', 'L', 'R', 'L', 'K', 'E', 'E', 'Q', 'K',
       'E', 'E', 'E', 'E', 'R', 'K', 'A', 'Q', 'E', 'E', 'Q', 'A', 'R',
       'R', 'E', 'H', 'E', 'E', 'Y', 'L', 'K', 'L', 'K', 'E', 'A', 'F',
       'V', 'V', 'E', 'E', 'E', 'G', 'V', 'S', 'E', 'T', 'M', 'T

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(min_df=0)

X__ = vec.fit_transform(random_seq_list)

#### LabelEncoder

In [14]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [15]:
X____ = encoder.fit_transform(random_seq_list)

##### Binarizer

In [ ]:
from sklearn.preprocessing import Binarizer

In [ ]:
binarizer = Binarizer()

X____b = binarizer.fit_transform(X____.reshape(-1,1))

In [ ]:
X____b # Doesn't work

##### Normalize

In [57]:
from sklearn.preprocessing import Normalizer

norm = Normalizer()

X____j = norm.fit_transform(X____.reshape(-1,1))

In [58]:
X____j

array([[0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [16]:
X____.max()

18

In [17]:
amino_acid_sequence = X____/18

In [1]:
amino_acid_sequence

NameError: name 'amino_acid_sequence' is not defined

#### Function that does the above for us (that works but isn't pratical)

In [7]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

def encode_sequence(sequence):

    encoder = LabelEncoder()

    random_sequence = np.array(list(sequence))
    random_encoded_sequence = encoder.fit_transform(random_sequence)
    random_encoded_sequence = random_encoded_sequence/random_encoded_sequence.max()
    
    return random_encoded_sequence



    
    # for n in len(df_updated["protein_sequence"]):
    #     random_sequence = df_updated.iloc[n]["protein_sequence"]
    #     random_sequence = np.array(list(random_sequence))
    #     random_encoded_sequence = encoder.fit_transform(random_sequence)
    #     random_encoded_sequence = random_encoded_sequence/random_encoded_sequence.max()

    #     return random_encoded_sequence

#### Lambda Expression?

In [ ]:
df_updated["protein_sequence_number"] = df_updated["protein_sequence"].map(lambda x: encode_sequence(x))

### Engineering Features

#### Finding Mean

In [19]:
from statistics import mean

In [57]:
df_updated

,seq_id,protein_sequence,pH,tm,average
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,hello
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,hello
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,hello
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,hello
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,hello
...,...,...,...,...,...
28976,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,hello
28977,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,hello
28978,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,hello
28979,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,hello


In [55]:
sequence = df_updated["protein_sequence"][1]

In [56]:
mean(sequence)

TypeError: can't convert type 'str' to numerator/denominator

In [48]:
df_updated["average"] = "hello"

In [50]:
df_updated

,seq_id,protein_sequence,pH,tm,average
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,hello
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,hello
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,hello
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,hello
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,hello
...,...,...,...,...,...
28976,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,hello
28977,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,hello
28978,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,hello
28979,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,hello


In [ ]:
df_updated["average"] = [mean(v) for v in df_updated["protein_sequence"].values]

In [52]:
df_updated

,seq_id,protein_sequence,pH,tm,average
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,hello
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,hello
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,hello
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,hello
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,hello
...,...,...,...,...,...
28976,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,hello
28977,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,hello
28978,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,hello
28979,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,hello


#### Length Column

In [2]:
df_updated["len"] = df_updated["protein_sequence"].str.len()

NameError: name 'df_updated' is not defined

#### ProteinAnalysis from biopython

In [62]:
df_updated

,seq_id,protein_sequence,pH,tm,average,molecular_weight,aromaticity
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,hello,36320.7244,0.058651
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,hello,32837.9931,0.041958
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,hello,53428.8034,0.080483
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,hello,29475.5996,0.071698
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,hello,158761.9814,0.065472
...,...,...,...,...,...,...,...
28976,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,hello,61997.6230,0.089253
28977,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,hello,52637.6897,0.087420
28978,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,hello,14203.9225,0.117188
28979,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,hello,64367.8724,0.074199


In [30]:
df_updated.corr()

,seq_id,pH,tm,average,len
seq_id,1.000000,-0.009309,-0.002663,-0.023293,-0.017527
pH,-0.009309,1.000000,0.027573,-0.087561,0.107266
tm,-0.002663,0.027573,1.000000,-0.185351,-0.137504
average,-0.023293,-0.087561,-0.185351,1.000000,0.067922
len,-0.017527,0.107266,-0.137504,0.067922,1.000000


In [9]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [10]:
df_updated["molecular_weight"] = df_updated["protein_sequence"].apply(lambda x: ProteinAnalysis(x).molecular_weight())

In [11]:
df_updated["aromaticity"] = df_updated["protein_sequence"].apply(lambda x: ProteinAnalysis(x).aromaticity())

In [12]:
df_updated["secondary__structure_fraction"] = df_updated["protein_sequence"].apply(lambda x: ProteinAnalysis(x).secondary_structure_fraction())

In [13]:
df_updated["instablity_index"] = df_updated["protein_sequence"].apply(lambda x: ProteinAnalysis(x).instability_index())

In [16]:
df_updated

,protein_sequence,pH,tm,molecular_weight,aromaticity,secondary__structure_fraction,instablity_index
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,36320.7244,0.058651,"(0.31671554252199413, 0.21114369501466276, 0.3...",28.389736
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,32837.9931,0.041958,"(0.21328671328671328, 0.16083916083916086, 0.3...",65.108392
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,53428.8034,0.080483,"(0.20925553319919515, 0.2676056338028169, 0.21...",35.089336
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,29475.5996,0.071698,"(0.2679245283018868, 0.21509433962264152, 0.29...",50.502264
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,158761.9814,0.065472,"(0.2715368711233632, 0.2929014472777395, 0.206...",45.668808
...,...,...,...,...,...,...,...
28976,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,61997.6230,0.089253,"(0.2877959927140255, 0.2422586520947177, 0.224...",48.604026
28977,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,52637.6897,0.087420,"(0.32409381663113, 0.21748400852878463, 0.2707...",35.285096
28978,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,14203.9225,0.117188,"(0.265625, 0.25, 0.2578125)",44.198437
28979,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,64367.8724,0.074199,"(0.2883642495784148, 0.26812816188870153, 0.25...",36.618398


In [17]:
df_updated.corr()

,pH,tm,molecular_weight,aromaticity,instablity_index
pH,1.000000,0.027573,0.105719,0.004668,0.147854
tm,0.027573,1.000000,-0.140078,-0.006742,-0.111089
molecular_weight,0.105719,-0.140078,1.000000,-0.106528,0.251000
aromaticity,0.004668,-0.006742,-0.106528,1.000000,-0.198232
instablity_index,0.147854,-0.111089,0.251000,-0.198232,1.000000


In [15]:
df_updated.drop("seq_id", axis = 1, inplace = True)

##### Column Stuff

In [18]:
import re

In [19]:
search_amino=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
for amino_acid in search_amino:
     df_updated[amino_acid]=df_updated['protein_sequence'].str.count(amino_acid,re.I)

In [21]:
df_updated

,protein_sequence,pH,tm,molecular_weight,aromaticity,secondary__structure_fraction,instablity_index,A,C,D,...,M,N,P,Q,R,S,T,V,W,Y
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,36320.7244,0.058651,"(0.31671554252199413, 0.21114369501466276, 0.3...",28.389736,45,1,13,...,8,5,18,6,25,11,14,37,4,3
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,32837.9931,0.041958,"(0.21328671328671328, 0.16083916083916086, 0.3...",65.108392,28,0,10,...,2,6,8,22,30,14,12,13,3,3
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,53428.8034,0.080483,"(0.20925553319919515, 0.2676056338028169, 0.21...",35.089336,50,9,27,...,6,15,20,25,31,33,30,30,3,16
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,29475.5996,0.071698,"(0.2679245283018868, 0.21509433962264152, 0.29...",50.502264,20,5,19,...,2,9,16,9,10,16,19,14,3,4
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,158761.9814,0.065472,"(0.2715368711233632, 0.2929014472777395, 0.206...",45.668808,86,14,78,...,31,65,128,54,63,148,120,124,16,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28976,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,61997.6230,0.089253,"(0.2877959927140255, 0.2422586520947177, 0.224...",48.604026,33,12,38,...,13,24,25,24,42,33,18,42,13,18
28977,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,52637.6897,0.087420,"(0.32409381663113, 0.21748400852878463, 0.2707...",35.285096,37,5,21,...,14,19,19,16,25,37,26,34,5,14
28978,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,14203.9225,0.117188,"(0.265625, 0.25, 0.2578125)",44.198437,13,1,7,...,7,5,6,8,3,10,6,7,4,4
28979,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,64367.8724,0.074199,"(0.2883642495784148, 0.26812816188870153, 0.25...",36.618398,47,5,34,...,26,25,31,12,25,51,32,48,3,18


#### Do all of that to the test data

In [23]:
df_updated

,protein_sequence,pH,tm,molecular_weight,aromaticity,secondary__structure_fraction,instablity_index,A,C,D,...,M,N,P,Q,R,S,T,V,W,Y
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,36320.7244,0.058651,"(0.31671554252199413, 0.21114369501466276, 0.3...",28.389736,45,1,13,...,8,5,18,6,25,11,14,37,4,3
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,32837.9931,0.041958,"(0.21328671328671328, 0.16083916083916086, 0.3...",65.108392,28,0,10,...,2,6,8,22,30,14,12,13,3,3
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,53428.8034,0.080483,"(0.20925553319919515, 0.2676056338028169, 0.21...",35.089336,50,9,27,...,6,15,20,25,31,33,30,30,3,16
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,29475.5996,0.071698,"(0.2679245283018868, 0.21509433962264152, 0.29...",50.502264,20,5,19,...,2,9,16,9,10,16,19,14,3,4
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,158761.9814,0.065472,"(0.2715368711233632, 0.2929014472777395, 0.206...",45.668808,86,14,78,...,31,65,128,54,63,148,120,124,16,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28976,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,61997.6230,0.089253,"(0.2877959927140255, 0.2422586520947177, 0.224...",48.604026,33,12,38,...,13,24,25,24,42,33,18,42,13,18
28977,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,52637.6897,0.087420,"(0.32409381663113, 0.21748400852878463, 0.2707...",35.285096,37,5,21,...,14,19,19,16,25,37,26,34,5,14
28978,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,14203.9225,0.117188,"(0.265625, 0.25, 0.2578125)",44.198437,13,1,7,...,7,5,6,8,3,10,6,7,4,4
28979,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,64367.8724,0.074199,"(0.2883642495784148, 0.26812816188870153, 0.25...",36.618398,47,5,34,...,26,25,31,12,25,51,32,48,3,18


In [26]:
df_updated.shape

(28981, 27)

In [24]:
df_test

,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


In [27]:
df_test.drop("data_source", axis = 1, inplace=True)

In [28]:
def feature_engineer(dataset1 = None, dataset2 = None):
    # Length of protein sequence
    dataset1["len"] = dataset1["protein_sequence"].str.len()
    dataset2["len"] = dataset2["protein_sequence"].str.len()

    # MW
    dataset1["molecular_weight"] = dataset1["protein_sequence"].apply(lambda x: ProteinAnalysis(x).molecular_weight())
    dataset2["molecular_weight"] = dataset2["protein_sequence"].apply(lambda x: ProteinAnalysis(x).molecular_weight())

    # Ar
    dataset1["aromaticity"] = dataset1["protein_sequence"].apply(lambda x: ProteinAnalysis(x).aromaticity())
    dataset2["aromaticity"] = dataset2["protein_sequence"].apply(lambda x: ProteinAnalysis(x).aromaticity())

    # SSF
    dataset1["secondary__structure_fraction"] = dataset1["protein_sequence"].apply(lambda x: ProteinAnalysis(x).secondary_structure_fraction())
    dataset2["secondary__structure_fraction"] = dataset2["protein_sequence"].apply(lambda x: ProteinAnalysis(x).secondary_structure_fraction())

    # II
    dataset1["instablity_index"] = dataset1["protein_sequence"].apply(lambda x: ProteinAnalysis(x).instability_index())
    dataset2["instablity_index"] = dataset2["protein_sequence"].apply(lambda x: ProteinAnalysis(x).instability_index())

    # Column stuff for amino acid
    search_amino=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

    for amino_acid in search_amino:
        dataset1[amino_acid]= dataset1['protein_sequence'].str.count(amino_acid,re.I)

    for amino_acid in search_amino:
        dataset2[amino_acid]= dataset2['protein_sequence'].str.count(amino_acid,re.I)


    train_data = dataset1
    test_data = dataset2
    return train_data, test_data
